In [73]:
import requests
import random
import pandas as pd
import time

In [74]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept-Language": 'en-US,en;q=0.9',
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://shopee.vn/",
    "From": "",
    "af-ac-enc-dat": "",
    "x-api-source": "pc"
}

base_image_url = "https://down-vn.img.susercontent.com/file/"


### EXTRACT GROUP

In [75]:
URL = "https://shopee.vn/api/v4/official_shop/get_categories"
PARAMS = { 
}

In [76]:
response = requests.get(URL, headers=HEADERS, params=PARAMS)
time.sleep(random.uniform(3.2, 8.7))
if response.status_code == 200:
    data = response.json()["data"]["categories"]
    group_data = []
    for group in data:
        group_id = group["category_id"]
        
        if group_id in [11035954, 11036030]:  
            name = group["name"]
            image = group["image"]
            group_data.append({"group_id": group_id, "name": name, "image": image})
    group = pd.DataFrame(group_data)
    print(f"Total group fetched: {len(group)}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Total group fetched: 2


### TRANSFORM GROUP

In [77]:
group["image_url"] = base_image_url + group["image"]
group.columns

Index(['group_id', 'name', 'image', 'image_url'], dtype='object')

In [78]:
group = group.drop(columns=["image"])

In [79]:
print(f"Total group filtered: {len(group)}")
group.head()

Total group filtered: 2


,group_id,name,image_url
0,11035954,Máy tính & Laptop,https://down-vn.img.susercontent.com/file/c3f3...
1,11036030,Điện Thoại và Phụ Kiện,https://down-vn.img.susercontent.com/file/3123...


In [87]:
group.to_csv("group.csv", index=False)

### EXTRACT SHOP

In [80]:
def get_shop_data(group_id):
    PARAMS = {
        "need_zhuyin": 0,
        "category_id": group_id
    }
    
    URL = "https://shopee.vn/api/v4/official_shop/get_shops_by_category"
    response = requests.get(URL, params=PARAMS)
    data = response.json()
    return data

In [81]:
shop_data = []

for group_id in group['group_id']:
    data = get_shop_data(group_id)
    group_id_res = data["data"]["cat_id"]
    brands = data["data"]["brands"]

    for brand in brands:
        brand_ids = brand["brand_ids"]

        for shop in brand_ids:
            username = shop.get("username", "")
            brand_name = shop.get("brand_name", "")
            shopid = shop.get("shopid", "")
            logo = shop.get("logo", "")
            logo_pc = shop.get("logo_pc", "")
            shop_collection_id = shop.get("shop_collection_id", "")
            ctime = shop.get("ctime", "")
            brand_label = shop.get("brand_label", "")

            shop_data.append({
                "group_id": group_id_res,
                "username": username,
                "brand_name": brand_name,
                "shop_id": shopid,
                "logo": logo,
                "logo_pc": logo_pc,
                "shop_collection_id": shop_collection_id,
                "ctime": ctime,
                "brand_label": brand_label
            })
    print(f"Group ID: {group_id_res} - Total shop: {len(shop_data)}")
    time.sleep(random.uniform(3.2, 8.7))
    
shops = pd.DataFrame(shop_data)
shops

Group ID: 11035954 - Total shop: 89
Group ID: 11036030 - Total shop: 197


,group_id,username,brand_name,shop_id,logo,logo_pc,shop_collection_id,ctime,brand_label
0,11035954,acer_anphat_store,Acer An Phát,780974114,vn-50009109-0ffb44fe08f981895c51a2fa3d05acff,vn-50009109-d7c2da2a0d77f0829f1fa22007963e3d,0,1698399378,0
1,11035954,acer_flagshipstore,Acer Flagship Store,333430029,3c828a89fcd3f8d4649b5d661e2f4ab5,5ed287b3a5f2076ba13f46aee0b0cf2e,0,1620981896,0
2,11035954,acer_flagship_hch_store,Acer Hàng Chính Hiệu,106066556,3c828a89fcd3f8d4649b5d661e2f4ab5,5ed287b3a5f2076ba13f46aee0b0cf2e,0,1620981896,0
3,11035954,thietbiadcomputer,AD,107559320,36d0873e376ca24ca2c5d2425d646dbd,6a6587ae8a1b6f5a58dd5e2fb0cad3bb,0,1620981896,0
4,11035954,tinhocvienthong,ADATA,8467305,48bd52f6bf8c10b070685ccedf4cc5da,6ba768709e5988e31a63e855faf5da98,0,1620981896,0
...,...,...,...,...,...,...,...,...,...
192,11036030,yoobao_flagship_zashop,Yoobao HCM,142777053,73a6e8aea0c7a0ec49e62e20eaf2c69e,630241110ba9ac1fc4c3a872dea18ae9,0,1620981895,0
193,11036030,yoobao_official_store,Yoobao HN,74291781,58476efa4dee3a156892268b5942d37c,d0944399fcef43efdc499cdaabf711ad,0,1620981895,0
194,11036030,zagg_flagship_store,Zagg,164018338,47e0896b81e463a83c2280b5cdf072fb,b550903998a0f05506b4c5d2ddb73804,0,1620981895,0
195,11036030,zeelot_official_store,ZMI Official Store,346553939,a3f3313f03ab50d0e933ace433e1cc27,9ee9b74d46e04613e5f5eea83a3bcb54,0,1620981896,0


In [86]:
shops.to_csv("shops.csv", index=False)

### TRANSFORM SHOP

In [82]:
shops["creation_time"] = pd.to_datetime(shops['ctime'], unit='s')
shops["shop_logo_url"] = base_image_url + shops["logo_pc"]
shops.columns
shops.head()

,group_id,username,brand_name,shop_id,logo,logo_pc,shop_collection_id,ctime,brand_label,creation_time,shop_logo_url
0,11035954,acer_anphat_store,Acer An Phát,780974114,vn-50009109-0ffb44fe08f981895c51a2fa3d05acff,vn-50009109-d7c2da2a0d77f0829f1fa22007963e3d,0,1698399378,0,2023-10-27 09:36:18,https://down-vn.img.susercontent.com/file/vn-5...
1,11035954,acer_flagshipstore,Acer Flagship Store,333430029,3c828a89fcd3f8d4649b5d661e2f4ab5,5ed287b3a5f2076ba13f46aee0b0cf2e,0,1620981896,0,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/5ed2...
2,11035954,acer_flagship_hch_store,Acer Hàng Chính Hiệu,106066556,3c828a89fcd3f8d4649b5d661e2f4ab5,5ed287b3a5f2076ba13f46aee0b0cf2e,0,1620981896,0,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/5ed2...
3,11035954,thietbiadcomputer,AD,107559320,36d0873e376ca24ca2c5d2425d646dbd,6a6587ae8a1b6f5a58dd5e2fb0cad3bb,0,1620981896,0,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/6a65...
4,11035954,tinhocvienthong,ADATA,8467305,48bd52f6bf8c10b070685ccedf4cc5da,6ba768709e5988e31a63e855faf5da98,0,1620981896,0,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/6ba7...


In [83]:
shops = shops.drop(columns=["logo_pc", "logo", "shop_collection_id", "brand_label", "ctime"])
shops.columns
shops.head(10)

,group_id,username,brand_name,shop_id,creation_time,shop_logo_url
0,11035954,acer_anphat_store,Acer An Phát,780974114,2023-10-27 09:36:18,https://down-vn.img.susercontent.com/file/vn-5...
1,11035954,acer_flagshipstore,Acer Flagship Store,333430029,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/5ed2...
2,11035954,acer_flagship_hch_store,Acer Hàng Chính Hiệu,106066556,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/5ed2...
3,11035954,thietbiadcomputer,AD,107559320,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/6a65...
4,11035954,tinhocvienthong,ADATA,8467305,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/6ba7...
5,11035954,akkovn,Akko Gear,283419910,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/8b60...
6,11035954,anphat_officialstore,An Phát Official Store,5526488,2022-01-05 09:44:09,https://down-vn.img.susercontent.com/file/96de...
7,11035954,apple_flagship_store,Apple,88201679,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/9833...
8,11035954,asus_flagship_hch_store,Asus Hàng Chính Hiệu,139249594,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/63a0...
9,11035954,anphatpc_hcm_store,Asus An Phát,292808202,2021-05-14 08:44:56,https://down-vn.img.susercontent.com/file/793e...


### EXTRACT SHOP BASE

In [85]:
group_ids = shops["group_id"].unique()

for gr_id in group_ids:
    gr_data = shops[shops['group_id'] == gr_id]

    shop_details = []

    for idx, row in gr_data.iterrows():
        username = row['username']
        shopid = row['shop_id']

        URL = "https://shopee.vn/api/v4/shop/get_shop_base"
        PARAMS = {
            "entry_point": "",
            "need_cancel_rate": "true",
            "request_source": "shop_home_page",
            "username": username,
            "version": "2",
            "shopid": shopid
        }
        
        response = requests.get(URL, headers=HEADERS, params=PARAMS)
        time.sleep(random.uniform(3.2, 8.7))

        if response.status_code == 200:
            print(f"Success - {username}: {response.status_code}")
            data = response.json()['data']
            userid = data['userid']
            follower_count = data['follower_count']
            total_avg_star = data['account']['total_avg_star']
            item_count = data['item_count']
            rating_normal = data['shop_rating']['rating_normal']
            rating_bad = data['shop_rating']['rating_bad']
            rating_good = data['shop_rating']['rating_good']
            response_time = data['response_time']
            ctime = data['ctime']

            shop_details.append({
                'group_id': row['group_id'],
                'username': row['username'],
                'brand_name': row['brand_name'],
                'shop_id': row['shop_id'],
                'creation_time': row['creation_time'],
                'shop_logo_url': row['shop_logo_url'],
                'userid': userid,
                'follower_count': follower_count,
                'total_avg_star': total_avg_star,
                'item_count': item_count,
                'rating_normal': rating_normal,
                'rating_bad': rating_bad,
                'rating_good': rating_good,
                'response_time': response_time,
                'ctime': ctime
            })
        else:
            print(f"Error - {username}: {response.status_code} - https://shopee.vn/api/v4/shop/get_shop_base?entry_point=&need_cancel_rate=true&request_source=shop_home_page&username={username}&version=2&shopid={shopid}")

    # Create a DataFrame for the current category_id
    shop_details_df = pd.DataFrame(shop_details)

    # Save the DataFrame to a CSV file with the desired file name format
    file_name = f"shop_base_category_{gr_id}.csv"
    shop_details_df.to_csv(file_name, index=False)

    print(f"Data for group_id {gr_id} saved to {file_name}")

Error - acer_anphat_store: 403 - https://shopee.vn/api/v4/shop/get_shop_base?entry_point=&need_cancel_rate=true&request_source=shop_home_page&username=acer_anphat_store&version=2&shopid=780974114
Error - acer_flagshipstore: 403 - https://shopee.vn/api/v4/shop/get_shop_base?entry_point=&need_cancel_rate=true&request_source=shop_home_page&username=acer_flagshipstore&version=2&shopid=333430029
Error - acer_flagship_hch_store: 403 - https://shopee.vn/api/v4/shop/get_shop_base?entry_point=&need_cancel_rate=true&request_source=shop_home_page&username=acer_flagship_hch_store&version=2&shopid=106066556
Error - thietbiadcomputer: 403 - https://shopee.vn/api/v4/shop/get_shop_base?entry_point=&need_cancel_rate=true&request_source=shop_home_page&username=thietbiadcomputer&version=2&shopid=107559320
Error - tinhocvienthong: 403 - https://shopee.vn/api/v4/shop/get_shop_base?entry_point=&need_cancel_rate=true&request_source=shop_home_page&username=tinhocvienthong&version=2&shopid=8467305
Error - akko